In [46]:
import sys
sys.path.insert(0, 'C:/Users/Diyahir/Documents/GitHub/StockBot/playground')
sys.path.insert(0, 'C:/Users/Diyahir/Documents/GitHub/StockBot/data/price')
from price_data import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import keras

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [28]:
import utils
import playground

In [128]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.

model.add(Dense(5, input_dim=94))
model.add(Activation('sigmoid'))
model.add(Dense(5))
model.add(Activation('sigmoid'))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [129]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_42 (Dense)             (None, 5)                 475       
_________________________________________________________________
activation_40 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_43 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 3)                 18        
_________________________________________________________________
activation_42 (Activation)   (None, 3)                 0         
Total params: 523
Trainable params: 523
Non-trainable params: 0
_________________________________________________________________


In [92]:
[a.shape for a in model.get_weights()]

[(94, 5), (5,), (5, 5), (5,), (5, 5), (5,)]

In [19]:
weights = model.get_weights()
print(weights)

[array([[-0.05978256, -0.45252523,  0.53486407,  0.07062006,  0.22514123],
       [-0.18938959, -0.19302922,  0.58747983, -0.11706644, -0.0781405 ],
       [ 0.56848991, -0.74859679,  0.40087152,  0.5254792 ,  0.38632143],
       [ 0.04639471,  0.22290057, -0.69508862,  0.14180815, -0.31835631],
       [-0.50300145, -0.72693658,  0.39375544,  0.44648552, -0.23415983]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.], dtype=float32), array([[-0.08577734,  0.63018656, -0.15444916, -0.26847726, -0.08295894],
       [ 0.27433228,  0.29336226, -0.39539871,  0.72215164,  0.21706414],
       [-0.22939903, -0.51345593, -0.57624894,  0.55744743, -0.3626824 ],
       [-0.13136142, -0.47897577, -0.67092496,  0.74631298,  0.30880344],
       [-0.28767699,  0.50821519,  0.65140045,  0.11499119, -0.09550577]], dtype=float32), array([ 0.,  0.,  0.,  0.,  0.], dtype=float32), array([[-0.8629303 ,  0.16733009,  0.48791009],
       [-0.67118776, -0.74493951, -0.68093467],
       [-0.40408018, -0.2969950

In [130]:
bot_tester = playground.TradingBot(1000,'FB',model)

In [87]:
fb_price_data = get_google_finance_intraday('FB', period=300, days=70)
fb_price_data.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2018-04-13 08:30:00,164.6000,164.65,164.530,164.58,265787.0
2018-04-13 08:35:00,165.1500,165.45,164.600,164.65,760116.0
2018-04-13 08:40:00,165.3100,165.42,164.900,165.15,540410.0
2018-04-13 08:45:00,165.0650,165.57,164.801,165.33,758604.0
2018-04-13 08:50:00,164.9201,165.60,164.740,165.04,597924.0


In [88]:
fb_sentiment_data = playground.get_company_tweets('C:/Users/Diyahir/Documents/GitHub/StockBot/data/twitter/historical.csv' ,'(F|f)acebook')
print(fb_sentiment_data)

C:/Users/Diyahir/Documents/GitHub/StockBot/playground\playground.py:100: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  company_tweets = data.loc[data['Texts'].str.contains(company_regex, na=False)]


                                Dates  Sentiment                  Datetime
8994   Tue Jun 27 13:02:00 +0000 2017     0.2023 2017-06-27 13:02:00+00:00
8781   Wed Jul 26 18:38:35 +0000 2017     0.0000 2017-07-26 18:38:35+00:00
8778   Wed Jul 26 21:06:09 +0000 2017     0.5106 2017-07-26 21:06:09+00:00
8724   Wed Aug 02 16:56:36 +0000 2017     0.4404 2017-08-02 16:56:36+00:00
8718   Wed Aug 02 19:00:15 +0000 2017     0.0000 2017-08-02 19:00:15+00:00
8705   Thu Aug 03 16:50:06 +0000 2017    -0.5132 2017-08-03 16:50:06+00:00
8653   Wed Aug 09 21:30:11 +0000 2017    -0.5267 2017-08-09 21:30:11+00:00
8649   Thu Aug 10 03:59:38 +0000 2017    -0.3724 2017-08-10 03:59:38+00:00
8635   Thu Aug 10 16:50:06 +0000 2017     0.3182 2017-08-10 16:50:06+00:00
8618   Fri Aug 11 14:50:06 +0000 2017     0.0000 2017-08-11 14:50:06+00:00
8599   Mon Aug 14 15:40:09 +0000 2017     0.3818 2017-08-14 15:40:09+00:00
8558   Thu Aug 17 13:10:07 +0000 2017    -0.6124 2017-08-17 13:10:07+00:00
8456   Tue Sep 05 14:16:5

In [150]:
dataslice = playground.evaluate_fitness(bot_tester,fb_price_data,fb_sentiment_data)
print(model.predict(dataslice))

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy
Tried to buy

ValueError: Error when checking : expected dense_42_input to have 2 dimensions, but got array with shape ()

In [124]:
dataslice = dataslice.reshape(1,94)

In [126]:
print(dataslice.shape)
print(model.predict(dataslice))

(1, 94)
[[ 0.2892921   0.32634753  0.1012408   0.15005812  0.13306148]]


In [10]:
mutated_weights = utils.create_offspring(weights,weights)
print(mutated_weights.neural_net)

[array([ 0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  0.,  0.,  0.])]


In [ ]:
print